## Protótipo Dashboard 
Neste protótipo, iremos mostrar quais dados e visualizações estarão presentes no dashboard WEB

<img src="https://i.gifer.com/7l6N.gif" width="400" height="400">/

### Import das bibliotecas

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

from models import Cliente, Local, Camera, Contagem

CLIENTE_ID = 1

### Conexão com o banco

In [2]:
DB_CONN_STRING = 'postgres://postgres:@localhost:5432/peoplecounter'

engine = create_engine(DB_CONN_STRING, echo=True)

### Nosso cliente cadastrado

In [42]:
query = "SELECT nome AS nome_cliente FROM clientes"

clientes = pd.read_sql(query, engine)

2018-10-24 13:58:43,174 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2018-10-24 13:58:43,176 INFO sqlalchemy.engine.base.Engine {'name': 'SELECT nome AS nome_cliente FROM clientes'}
2018-10-24 13:58:43,177 INFO sqlalchemy.engine.base.Engine SELECT nome AS nome_cliente FROM clientes
2018-10-24 13:58:43,178 INFO sqlalchemy.engine.base.Engine {}


In [43]:
clientes

,nome_cliente
0,DB SUPERMERCADOS


### Estabelecimentos do cliente cadastrados

In [44]:
query = "SELECT cl.nome AS cliente_nome, l.nome AS local_nome, l.cep, l.endereco, l.cidade, l.estado \
        FROM locais l \
        RIGHT JOIN clientes cl ON l.cliente_id = cl.cliente_id \
        WHERE l.cliente_id = {0} \
        ORDER BY l.local_id".format(CLIENTE_ID)

locais = pd.read_sql(query, engine)

2018-10-24 13:58:55,847 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2018-10-24 13:58:55,848 INFO sqlalchemy.engine.base.Engine {'name': 'SELECT cl.nome AS cliente_nome, l.nome AS local_nome, l.cep, l.endereco, l.cidade, l.estado         FROM locais l         RIGHT JOIN clientes cl ON l.cliente_id = cl.cliente_id         WHERE l.cliente_id = 1         ORDER BY l.local_id'}
2018-10-24 13:58:55,850 INFO sqlalchemy.engine.base.Engine SELECT cl.nome AS cliente_nome, l.nome AS local_nome, l.cep, l.endereco, l.cidade, l.estado         FROM locais l         RIGHT JOIN clientes cl ON l.cliente_id = cl.cliente_id         WHERE l.cliente_id = 1         ORDER BY l.local_id
2018-10-24 13:58:55,851 INFO sqlalchemy.engine.base.Engine {}


In [45]:
locais

,cliente_nome,local_nome,cep,endereco,cidade,estado
0,DB SUPERMERCADOS,DB CIDADE NOVA,69095-000,"AV. NOEL NUTELS, 1762 - CIDADE NOVA",MANAUS,AMAZONAS
1,DB SUPERMERCADOS,DB PARAIBA,69057-021,"AV. JORNALISTA HUMBERTO CALDERADO FILHO, 1.128...",MANAUS,AMAZONAS


### Cameras de cada estabelecimento do cliente cadastradas

In [46]:
query = "SELECT cl.nome AS cliente_nome, l.nome AS local_nome, cam.nome AS camera_nome \
        FROM cameras cam \
        RIGHT JOIN locais l ON cam.local_id = l.local_id \
        RIGHT JOIN clientes cl ON l.cliente_id = cl.cliente_id \
        WHERE cl.cliente_id = {0} \
        ORDER BY cam.camera_id".format(CLIENTE_ID)

cameras = pd.read_sql(query, engine)

2018-10-24 13:59:06,553 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2018-10-24 13:59:06,554 INFO sqlalchemy.engine.base.Engine {'name': 'SELECT cl.nome AS cliente_nome, l.nome AS local_nome, cam.nome AS camera_nome         FROM cameras cam         RIGHT JOIN locais l ON cam.local_id = l.local_id         RIGHT JOIN clientes cl ON l.cliente_id = cl.cliente_id         WHERE cl.cliente_id = 1         ORDER BY cam.camera_id'}
2018-10-24 13:59:06,556 INFO sqlalchemy.engine.base.Engine SELECT cl.nome AS cliente_nome, l.nome AS local_nome, cam.nome AS camera_nome         FROM cameras cam         RIGHT JOIN locais l ON cam.local_id = l.local_id         RIGHT JOIN clientes cl ON l.cliente_id = cl.cliente_id         WHERE cl.cliente_id = 1         ORDER BY cam.camera_id
2018-10-24 13:59:06,556 INFO sqlalchemy.engine.base.Engine {}


In [47]:
cameras

,cliente_nome,local_nome,camera_nome
0,DB SUPERMERCADOS,DB CIDADE NOVA,CAMERA ENTRADA 1
1,DB SUPERMERCADOS,DB CIDADE NOVA,CAMERA ENTRADA 2
2,DB SUPERMERCADOS,DB PARAIBA,CAM_ENTRADA_1
3,DB SUPERMERCADOS,DB PARAIBA,CAM_ENTRADA_2


### Simulação de busca de local e camera por local pelo usuário na interface

Busca do local (estabelecimento) para o cliente 1 cadastrado 

In [56]:
busca_local = input("Digite o nome do estabelecimento: ")

Digite o nome do estabelecimento: cida


In [57]:
query = "SELECT cl.nome AS cliente_nome, l.nome AS local_nome, l.cep, l.endereco, l.cidade, l.estado \
        FROM locais l \
        RIGHT JOIN clientes cl ON l.cliente_id = cl.cliente_id \
        WHERE l.cliente_id = {0} AND l.nome LIKE '%%{1}%%'".format(CLIENTE_ID, busca_local.upper())

local_selecionado = pd.read_sql(query, engine)

2018-10-24 13:59:47,936 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2018-10-24 13:59:47,937 INFO sqlalchemy.engine.base.Engine {'name': "SELECT cl.nome AS cliente_nome, l.nome AS local_nome, l.cep, l.endereco, l.cidade, l.estado         FROM locais l         RIGHT JOIN clientes cl ON l.cliente_id = cl.cliente_id         WHERE l.cliente_id = 1 AND l.nome LIKE '%%CIDA%%'"}
2018-10-24 13:59:47,939 INFO sqlalchemy.engine.base.Engine SELECT cl.nome AS cliente_nome, l.nome AS local_nome, l.cep, l.endereco, l.cidade, l.estado         FROM locais l         RIGHT JOIN clientes cl ON l.cliente_id = cl.cliente_id         WHERE l.cliente_id = 1 AND l.nome LIKE '%%CIDA%%'
2018-10-24 13:59:47,940 INFO sqlalchemy.engine.base.Engine {}


#### Abstraímos que o usuário selecionou o local que foi resultado da busca

In [58]:
local_selecionado

,cliente_nome,local_nome,cep,endereco,cidade,estado
0,DB SUPERMERCADOS,DB CIDADE NOVA,69095-000,"AV. NOEL NUTELS, 1762 - CIDADE NOVA",MANAUS,AMAZONAS


Busca da câmera a partir do local selecionado

In [59]:
busca_camera = input("Digite o nome/numero da camera: ")

Digite o nome/numero da camera: 1


In [60]:
query = "SELECT cl.nome AS cliente_nome, l.nome AS local_nome, cam.nome AS camera_nome \
        FROM cameras cam \
        RIGHT JOIN locais l ON cam.local_id = l.local_id \
        RIGHT JOIN clientes cl ON l.cliente_id = cl.cliente_id \
        WHERE l.cliente_id = {0} AND l.cep = '{1}' AND cam.nome LIKE '%%{2}%%'".format(CLIENTE_ID, local_selecionado.cep[0], busca_camera.upper())
    
camera_selecionada = pd.read_sql(query, engine)

2018-10-24 13:59:52,176 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2018-10-24 13:59:52,177 INFO sqlalchemy.engine.base.Engine {'name': "SELECT cl.nome AS cliente_nome, l.nome AS local_nome, cam.nome AS camera_nome         FROM cameras cam         RIGHT JOIN locais l ON cam.local_id =  ... (5 characters truncated) ... al_id         RIGHT JOIN clientes cl ON l.cliente_id = cl.cliente_id         WHERE l.cliente_id = 1 AND l.cep = '69095-000' AND cam.nome LIKE '%%1%%'"}
2018-10-24 13:59:52,179 INFO sqlalchemy.engine.base.Engine SELECT cl.nome AS cliente_nome, l.nome AS local_nome, cam.nome AS camera_nome         FROM cameras cam         RIGHT JOIN locais l ON cam.local_id = l.local_id         RIGHT JOIN clientes cl ON l.cliente_id = cl.cliente_id         WHERE l.cliente_id = 1 AND l.cep = '69095-000' AND cam.nome LIKE '%%1%%'
2018-10-24 13:59:52,180 INFO sqlal

#### Abstraímos que o usuário selecionou a câmera que foi resultado da busca

In [61]:
camera_selecionada

,cliente_nome,local_nome,camera_nome
0,DB SUPERMERCADOS,DB CIDADE NOVA,CAMERA ENTRADA 1


### Contagem de pessoas por câmera
Considera-se que cada camera cadastrada representa uma entrada diferente para o establecimento.
Neste exemplo, veremos os dados de contagem da camera 1 do local 1 do nosso cliente cadastrado.

In [66]:
query = "SELECT cont.timestamp, cont.qtd_pessoas_in \
        FROM contagem cont \
        RIGHT JOIN cameras cam ON cont.camera_id = cam.camera_id\
        RIGHT JOIN locais l ON cam.local_id = l.local_id \
        RIGHT JOIN clientes cl ON l.cliente_id = cl.cliente_id \
        WHERE cl.cliente_id = {0} AND l.cep = '{1}' AND cam.nome = '{2}' \
        ORDER BY cont.contagem_id".format(CLIENTE_ID, local_selecionado.cep[0], camera_selecionada.camera_nome[0])

contagem = pd.read_sql(query, engine)

2018-10-24 14:01:10,507 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2018-10-24 14:01:10,509 INFO sqlalchemy.engine.base.Engine {'name': "SELECT cont.timestamp, cont.qtd_pessoas_in         FROM contagem cont         RIGHT JOIN cameras cam ON cont.camera_id = cam.camera_id        RIGHT J ... (79 characters truncated) ... liente_id = cl.cliente_id         WHERE cl.cliente_id = 1 AND l.cep = '69095-000' AND cam.nome = 'CAMERA ENTRADA 1'         ORDER BY cont.contagem_id"}
2018-10-24 14:01:10,510 INFO sqlalchemy.engine.base.Engine SELECT cont.timestamp, cont.qtd_pessoas_in         FROM contagem cont         RIGHT JOIN cameras cam ON cont.camera_id = cam.camera_id        RIGHT JOIN locais l ON cam.local_id = l.local_id         RIGHT JOIN clientes cl ON l.cliente_id = cl.cliente_id         WHERE cl.cliente_id = 1 AND l.cep = '69095-000' AND cam.nome = 'CAMERA ENTR

In [67]:
contagem.head()

,timestamp,qtd_pessoas_in
0,2018-10-24 00:43:03.003574,2
1,2018-10-24 00:43:06.017777,1
2,2018-10-24 00:43:09.007212,4
3,2018-10-24 00:43:12.059917,5
4,2018-10-24 00:43:15.034008,6


### Resumo Estatístico

In [69]:
contagem.describe()

,qtd_pessoas_in
count,83.000000
mean,1.072289
std,1.386196
min,0.000000
25%,0.000000
50%,1.000000
75%,1.500000
max,6.000000
